# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

> As a used car dealership, it is important to us to understand what an appropriate market price is for the cars we sell. This dataset contains a large number of used cars with various details about them and how much they sold for. Using this large dataset, we can construct a theoretical model of how much each of these different details contributes to the market value of a car. This model can then, hopefully, be used to predict with decent precision the expected value of other cars that are not in the dataset and have not been sold yet. In doing so, the prices we set can be as attuned as possible to the details of the market, maximizing business profits.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [1]:
import pandas as pd
import numpy as np

In [2]:
vehicles = pd.read_csv('data/vehicles.csv')

In [3]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [4]:
vehicles.condition.unique()

array([nan, 'good', 'excellent', 'fair', 'like new', 'new', 'salvage'],
      dtype=object)

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [58]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [60]:
vehicles_cleaned = vehicles.dropna(subset=['year', 'odometer']).fillna('unknown')

In [138]:
vehicles_cleaned2 = vehicles_cleaned.drop(vehicles_cleaned.query('price >= 400_000 or odometer > 500_000').index)

In [100]:
unknown_manu = vehicles_cleaned2.query('manufacturer == "unknown"')
individual_terms_in_model = pd.Series([item.lower() for sublist in unknown_manu['model'].str.split(' ').to_list() for item in sublist]).value_counts()

In [150]:
individual_terms_in_model.head(20)

scion            1894
4d               1159
hatchback        1007
international     968
freightliner      827
genesis           718
isuzu             653
im                644
sedan             622
hummer            522
g70               437
all               409
oldsmobile        409
npr               391
maserati          375
suzuki            329
saab              328
4300              306
tc                301
xb                293
Name: count, dtype: int64

In [151]:
mapping_dict = {'scion': 'scion', 'freightliner': 'freightliner', 'genesis': 'genesis', 'isuzu': 'isuzu', 'oldsmobile': 'oldsmobile',
                'maserati': 'maserati', 'suzuki': 'suzuki', 'saab': 'saab', 'corvette': 'corvette', 'chevorlet': 'chevrolet',
                'volkswagon': 'volkswagen', 'lamborghini': 'lamborghini', 'rolls royce': 'rolls royce'}

In [153]:
vehicles_cleaned3 = vehicles_cleaned2

for key, value in mapping_dict.items():
    vehicles_cleaned3.loc[vehicles_cleaned3[vehicles_cleaned3['model'].str.contains(key, case=False)].index, 'manufacturer'] = value



In [154]:
vehicles_cleaned3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419892 entries, 27 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            419892 non-null  int64  
 1   region        419892 non-null  object 
 2   price         419892 non-null  int64  
 3   year          419892 non-null  float64
 4   manufacturer  419892 non-null  object 
 5   model         419892 non-null  object 
 6   condition     419892 non-null  object 
 7   cylinders     419892 non-null  object 
 8   fuel          419892 non-null  object 
 9   odometer      419892 non-null  float64
 10  title_status  419892 non-null  object 
 11  transmission  419892 non-null  object 
 12  VIN           419892 non-null  object 
 13  drive         419892 non-null  object 
 14  size          419892 non-null  object 
 15  type          419892 non-null  object 
 16  paint_color   419892 non-null  object 
 17  state         419892 non-null  object 
dtypes: float

In [156]:
vehicles_cleaned4 = vehicles_cleaned3.drop(index=vehicles_cleaned3[vehicles_cleaned3['model'].str.contains('any and all', case=False)].index)
vehicles_cleaned4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419872 entries, 27 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            419872 non-null  int64  
 1   region        419872 non-null  object 
 2   price         419872 non-null  int64  
 3   year          419872 non-null  float64
 4   manufacturer  419872 non-null  object 
 5   model         419872 non-null  object 
 6   condition     419872 non-null  object 
 7   cylinders     419872 non-null  object 
 8   fuel          419872 non-null  object 
 9   odometer      419872 non-null  float64
 10  title_status  419872 non-null  object 
 11  transmission  419872 non-null  object 
 12  VIN           419872 non-null  object 
 13  drive         419872 non-null  object 
 14  size          419872 non-null  object 
 15  type          419872 non-null  object 
 16  paint_color   419872 non-null  object 
 17  state         419872 non-null  object 
dtypes: float

In [ ]:
empty_manus

In [53]:
lambo_str = 'lamborghini'
lamborghinis = vehicles4[vehicles4['model'].str.contains(lambo_str, case=False)]
vehicles4.loc[lamborghinis.index, 'manufacturer'] = lambo_str
vehicles4.query('manufacturer == "lamborghini"')

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
9080,7306378728,flagstaff / sedona,229500,2019.0,lamborghini,Lamborghini Urus,unknown,8 cylinders,gas,32922.0,clean,automatic,ZPBUA1ZL1KLA02237,4wd,compact,SUV,white,az
11339,7316288220,phoenix,109945,2004.0,lamborghini,Lamborghini Gallardo 6-Speed,excellent,10 cylinders,gas,57115.0,clean,manual,ZHWGU11M24LA01112,4wd,unknown,coupe,unknown,az
13050,7311968523,prescott,0,2021.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,74.0,clean,automatic,ZHWUF5ZF8MLA17048,rwd,unknown,coupe,unknown,az
16144,7311968117,tucson,0,2021.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,74.0,clean,automatic,ZHWUF5ZF8MLA17048,rwd,unknown,coupe,unknown,az
24221,7311967900,chico,0,2021.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,74.0,clean,automatic,ZHWUF5ZF8MLA17048,rwd,unknown,coupe,unknown,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376017,7315051233,houston,299991,2017.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,13004.0,clean,automatic,ZHWUC2ZF4HLA06366,rwd,unknown,coupe,blue,tx
382097,7311968287,san antonio,0,2021.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,74.0,clean,automatic,ZHWUF5ZF8MLA17048,rwd,unknown,coupe,unknown,tx
385995,7311968369,salt lake city,0,2021.0,lamborghini,Lamborghini Huracan,unknown,10 cylinders,gas,74.0,clean,automatic,ZHWUF5ZF8MLA17048,rwd,unknown,coupe,unknown,ut
415796,7306494891,appleton-oshkosh-FDL,229500,2019.0,lamborghini,Lamborghini Urus,unknown,8 cylinders,gas,32922.0,clean,automatic,ZPBUA1ZL1KLA02237,4wd,compact,SUV,white,wi


In [55]:
empty_manus = vehicles4.query('manufacturer == "unknown"')
empty_manus

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
97,7309271279,auburn,15990,2016.0,unknown,Scion iM Hatchback 4D,good,unknown,gas,29652.0,clean,other,JTNKARJEXGJ517925,fwd,unknown,hatchback,blue,al
122,7306198330,auburn,6800,2005.0,unknown,blue bird bus,excellent,6 cylinders,diesel,180000.0,clean,automatic,unknown,rwd,full-size,bus,yellow,al
135,7305150004,auburn,14990,2016.0,unknown,Scion iM Hatchback 4D,good,unknown,gas,65203.0,clean,other,JTNKARJE4GJ508198,fwd,unknown,hatchback,red,al
137,7305082143,auburn,2500,1966.0,unknown,1966 C-30 1 ton,good,6 cylinders,gas,47000.0,clean,manual,unknown,rwd,full-size,pickup,brown,al
155,7302642422,auburn,8990,2013.0,unknown,smart fortwo Passion Hatchback,good,unknown,gas,59072.0,clean,automatic,WMEEJ3BA2DK636540,rwd,unknown,coupe,silver,al
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426843,7302129327,wyoming,21999,2017.0,unknown,2017,unknown,unknown,gas,51000.0,clean,automatic,unknown,unknown,unknown,unknown,unknown,wy
426846,7302118268,wyoming,41999,2015.0,unknown,2015,unknown,unknown,gas,126000.0,clean,automatic,unknown,unknown,unknown,unknown,unknown,wy
426862,7301950980,wyoming,27590,2018.0,unknown,Genesis G80 3.8 Sedan 4D,good,6 cylinders,gas,36138.0,clean,other,KMHGN4JE4JU248193,unknown,unknown,sedan,black,wy
426863,7301949724,wyoming,25590,2017.0,unknown,Genesis G80 3.8 Sedan 4D,good,6 cylinders,gas,37608.0,clean,automatic,KMHGN4JE2HU183838,rwd,unknown,sedan,white,wy


In [57]:
vehicles.query('manufacturer == "scion"')

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state


In [36]:
non_null_models = vehicles.dropna(subset='model')
any_models = non_null_models[non_null_models['model'].str.contains('any and all', case=False)]

In [37]:
any_models.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 17752 to 369944
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            23 non-null     int64  
 1   region        23 non-null     object 
 2   price         23 non-null     int64  
 3   year          23 non-null     float64
 4   manufacturer  0 non-null      object 
 5   model         23 non-null     object 
 6   condition     6 non-null      object 
 7   cylinders     5 non-null      object 
 8   fuel          23 non-null     object 
 9   odometer      23 non-null     float64
 10  title_status  23 non-null     object 
 11  transmission  23 non-null     object 
 12  VIN           0 non-null      object 
 13  drive         4 non-null      object 
 14  size          4 non-null      object 
 15  type          4 non-null      object 
 16  paint_color   4 non-null      object 
 17  state         23 non-null     object 
dtypes: float64(2), int64(2), obje

In [38]:
any_models

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
17752,7310706844,fayetteville,100,2005.0,NaN,any and all,NaN,NaN,gas,1000.0,clean,manual,NaN,NaN,NaN,NaN,NaN,ar
20182,7309263368,little rock,0,2020.0,NaN,any and all,NaN,NaN,gas,50000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,ar
52305,7316411898,sacramento,10000,1998.0,NaN,any and all,NaN,NaN,diesel,129985.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,ca
101279,7310684595,lakeland,1000,2019.0,NaN,any and all,fair,other,gas,100000.0,clean,other,NaN,4wd,full-size,coupe,white,fl
101860,7301847442,lakeland,1000,2021.0,NaN,any and all,NaN,NaN,gas,123456.0,missing,other,NaN,NaN,NaN,NaN,NaN,fl
154138,7304826988,muncie / anderson,75,1900.0,NaN,any and all,NaN,NaN,other,10000.0,clean,other,NaN,NaN,NaN,NaN,NaN,in
164628,7316015924,"kansas city, MO",1,1960.0,NaN,any and all,NaN,NaN,other,1.0,clean,other,NaN,NaN,NaN,NaN,NaN,ks
194726,7316565620,battle creek,0,2007.0,NaN,any and all,NaN,12 cylinders,gas,12345.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,mi
201183,7313070883,grand rapids,0,1964.0,NaN,any and all,NaN,NaN,other,5.0,clean,other,NaN,NaN,NaN,NaN,NaN,mi
203547,7316566803,jackson,300,2010.0,NaN,any and all,new,NaN,gas,123346.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,mi


In [5]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [6]:
vehicles.nunique()

id              426880
region             404
price            15655
year               114
manufacturer        42
model            29649
condition            6
cylinders            8
fuel                 5
odometer        104870
title_status         6
transmission         3
VIN             118246
drive                3
size                 4
type                13
paint_color         12
state               51
dtype: int64

In [7]:
vehicles.type.value_counts()

type
sedan          87056
SUV            77284
pickup         43510
truck          35279
other          22110
coupe          19204
hatchback      16598
wagon          10751
van             8548
convertible     7731
mini-van        4825
offroad          609
bus              517
Name: count, dtype: int64

First, we can drop the id and VIN columns as they are simply unique identifiers that tell us nothing specific about the car. Then, we notice that there are 30k different models, which is far too many to incorporate in a linear regression model, so we will drop that column as well. The 'type' column is a reasonable and much more workable substitute.

In [13]:
vehicles_cleaned = vehicles.drop(['id', 'VIN', 'model', 'region', 'state'], axis='columns') \
.dropna(subset=['year', 'odometer']) \
.fillna('unknown') \
.rename(columns={'year': 'age'})

vehicles_cleaned['age'] = 2025 - vehicles_cleaned['age']

In [14]:
vehicles_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421344 entries, 27 to 426879
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         421344 non-null  int64  
 1   age           421344 non-null  float64
 2   manufacturer  421344 non-null  object 
 3   condition     421344 non-null  object 
 4   cylinders     421344 non-null  object 
 5   fuel          421344 non-null  object 
 6   odometer      421344 non-null  float64
 7   title_status  421344 non-null  object 
 8   transmission  421344 non-null  object 
 9   drive         421344 non-null  object 
 10  size          421344 non-null  object 
 11  type          421344 non-null  object 
 12  paint_color   421344 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 45.0+ MB


In [15]:
vehicles_cleaned.sample(10, random_state=42)

,price,age,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
218361,4500,16.0,chevrolet,good,6 cylinders,gas,136000.0,clean,automatic,fwd,full-size,sedan,red
202573,12997,11.0,ford,unknown,4 cylinders,gas,99437.0,clean,automatic,fwd,compact,van,white
146235,22500,94.0,ford,good,8 cylinders,gas,1500.0,clean,automatic,rwd,unknown,unknown,white
81067,8999,12.0,honda,excellent,unknown,gas,158208.0,clean,automatic,fwd,unknown,sedan,unknown
365494,1,75.0,chevrolet,excellent,8 cylinders,gas,31000.0,clean,automatic,rwd,unknown,coupe,red
838,49990,10.0,unknown,good,6 cylinders,diesel,203867.0,clean,automatic,rwd,unknown,unknown,white
293587,9995,12.0,mazda,unknown,unknown,gas,127012.0,clean,automatic,fwd,unknown,wagon,unknown
53687,3500,15.0,toyota,unknown,unknown,gas,200000.0,clean,automatic,unknown,unknown,unknown,unknown
81842,5500,21.0,unknown,good,8 cylinders,gas,138700.0,clean,manual,4wd,unknown,SUV,unknown
266505,3500,16.0,pontiac,good,4 cylinders,gas,196000.0,clean,automatic,4wd,mid-size,hatchback,blue


In [16]:
vehicles_encoded = pd.get_dummies(vehicles_cleaned)

In [17]:
vehicles_encoded.sample(5, random_state=42)

,price,age,odometer,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,...,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_unknown,paint_color_white,paint_color_yellow
218361,4500,16.0,136000.0,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
202573,12997,11.0,99437.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
146235,22500,94.0,1500.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
81067,8999,12.0,158208.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
365494,1,75.0,31000.0,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.